In [1]:
# === In[1]: базовые импорты ===
import os
import sys
import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Если ноутбук лежит в корне репозитория, чтобы импортировать backend:
ROOT_DIR = Path(".").resolve()
BACKEND_DIR = ROOT_DIR / "backend"

if str(BACKEND_DIR) not in sys.path:
    sys.path.append(str(BACKEND_DIR))

from pdf_processor.ocr_scripts.ocr_caller import process_pdf


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [2]:
# === In[2]: путь к файлу и запуск OCR ===

PDF_PATH = ROOT_DIR / "samples" / "invoice_example.pdf"  # поменяй на свой файл

assert PDF_PATH.exists(), f"Файл не найден: {PDF_PATH}"

ocr_result = process_pdf(str(PDF_PATH))


C:\Users\Uzer\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [3]:
# === In[3]: быстрый просмотр первой страницы ===

from pprint import pprint

print("Ключи верхнего уровня:", ocr_result.keys())
print("Количество страниц:", len(ocr_result["pages"]))

first_page = ocr_result["pages"][0]
print("Номер страницы:", first_page["page_number"])
print("Количество строк на странице:", len(first_page["lines"]))

print("\nПример первой строки (как список блоков):")
pprint(first_page["lines"][0][:5])  # первые 5 блоков первой строки


Ключи верхнего уровня: dict_keys(['pages'])
Количество страниц: 1
Номер страницы: 1
Количество строк на странице: 50

Пример первой строки (как список блоков):
[{'bounding_box': {'height': 66, 'width': 418, 'x': 711, 'y': 278},
  'text': 'ООО "Банк Точка\''},
 {'bounding_box': {'height': 33, 'width': 26, 'x': 1152, 'y': 297},
  'text': 'Г'},
 {'bounding_box': {'height': 63, 'width': 181, 'x': 1188, 'y': 281},
  'text': 'Москва'},
 {'bounding_box': {'height': 61, 'width': 120, 'x': 1683, 'y': 283},
  'text': 'БИК'},
 {'bounding_box': {'height': 61, 'width': 262, 'x': 1880, 'y': 283},
  'text': '044525104'}]


In [4]:
# === In[4]: функция для преобразования результата OCR в DataFrame ===

def ocr_result_to_dataframe(ocr_result: dict) -> pd.DataFrame:
    rows = []

    for page in ocr_result.get("pages", []):
        page_num = page["page_number"]
        for line_idx, line in enumerate(page["lines"]):
            # Вся строка целиком
            line_text = " ".join(block["text"] for block in line)

            for block_idx, block in enumerate(line):
                bb = block["bounding_box"]
                rows.append(
                    {
                        "page": page_num,
                        "line": line_idx,
                        "block": block_idx,
                        "text": block["text"],
                        "line_text": line_text,
                        "x": bb["x"],
                        "y": bb["y"],
                        "width": bb["width"],
                        "height": bb["height"],
                    }
                )

    df = pd.DataFrame(rows)
    return df


df_blocks = ocr_result_to_dataframe(ocr_result)
df_blocks.head()


,page,line,block,text,line_text,x,y,width,height
0,1,0,0,"ООО ""Банк Точка'","ООО ""Банк Точка' Г Москва БИК 044525104",711,278,418,66
1,1,0,1,Г,"ООО ""Банк Точка' Г Москва БИК 044525104",1152,297,26,33
2,1,0,2,Москва,"ООО ""Банк Точка' Г Москва БИК 044525104",1188,281,181,63
3,1,0,3,БИК,"ООО ""Банк Точка' Г Москва БИК 044525104",1683,283,120,61
4,1,0,4,044525104,"ООО ""Банк Точка' Г Москва БИК 044525104",1880,283,262,61


In [5]:
# === In[5]: агрегаты по строкам ===

df_lines = (
    df_blocks
    .groupby(["page", "line"], as_index=False)
    .agg(
        line_text=("line_text", "first"),
        n_blocks=("block", "count"),
        min_x=("x", "min"),
        min_y=("y", "min"),
        max_x=("x", lambda s: (s + df_blocks.loc[s.index, "width"]).max()),
        max_y=("y", lambda s: (s + df_blocks.loc[s.index, "height"]).max()),
    )
)

df_lines.head(10)


,page,line,line_text,n_blocks,min_x,min_y,max_x,max_y
0,1,0,"ООО ""Банк Точка' Г Москва БИК 044525104",5,711,278,2142,344
1,1,1,Сч. Ng 30101810745374525104,2,1679,354,2448,417
2,1,2,Банк получателя,1,715,416,1067,468
3,1,3,НОвЫе 40702810201500081796,2,348,454,2448,530
4,1,4,технопогии ИНН 7814529734 КПП 781001001 Сч. Ng,6,348,477,1853,538
5,1,5,WWWnт MAIL@NT,2,349,525,589,541
6,1,6,OОО 'НОВЫЕ ТЕХНОЛОГИИ',2,715,547,1422,603
7,1,7,Получатель,1,710,665,974,725
8,1,8,Счет на оплату Ng 650 от 15 сентября 2025 г,1,194,784,1946,905
9,1,9,"Поставщик ООО ""НОВЫЕ ТЕХНОЛОГИИ"", ИНН 78145297...",2,194,977,2677,1058


In [6]:
# === In[6]: поиск строк, где есть потенциальные суммы и ключевые слова ===

import re

money_pattern = re.compile(r"\d+[ ,.]?\d*\s?(RUB|₽|EUR|USD|BYN|KZT)?", re.IGNORECASE)

def contains_money(text: str) -> bool:
    return bool(money_pattern.search(text))

keywords = ["итог", "итого", "сумма", "total", "amount"]

def contains_keywords(text: str) -> bool:
    lower = text.lower()
    return any(k in lower for k in keywords)

df_lines["has_money"] = df_lines["line_text"].apply(contains_money)
df_lines["has_keyword"] = df_lines["line_text"].apply(contains_keywords)

interesting_lines = df_lines[df_lines["has_money"] | df_lines["has_keyword"]].copy()
interesting_lines.sort_values(["page", "line"], inplace=True)

interesting_lines[["page", "line", "line_text", "has_money", "has_keyword"]]


,page,line,line_text,has_money,has_keyword
0,1,0,"ООО ""Банк Точка' Г Москва БИК 044525104",True,False
1,1,1,Сч. Ng 30101810745374525104,True,False
3,1,3,НОвЫе 40702810201500081796,True,False
4,1,4,технопогии ИНН 7814529734 КПП 781001001 Сч. Ng,True,False
8,1,8,Счет на оплату Ng 650 от 15 сентября 2025 г,True,False
9,1,9,"Поставщик ООО ""НОВЫЕ ТЕХНОЛОГИИ"", ИНН 78145297...",True,False
11,1,11,"Заозёрная; дом 8, корпус 2, литера А; помещени...",True,False
12,1,12,"Покупатель АО ""МК ""ВЫСОТА"", ИНН 6672212600, КП...",True,False
13,1,13,(Заказчик): область; г: Екатеринбург ул. Лунач...,True,False
14,1,14,Основание: Счет 650 от 15.09.2025,True,False


In [8]:
# === In[7]: сохранение результата OCR в JSON ===

RESULT_DIR = ROOT_DIR / "media" / "ocr_results"
RESULT_DIR.mkdir(parents=True, exist_ok=True)

# Например, использовать имя PDF без расширения как id
doc_id = PDF_PATH.stem
result_file_path = RESULT_DIR / f"{doc_id}.json"

with open(result_file_path, "w", encoding="utf-8") as f:
    json.dump(ocr_result, f, ensure_ascii=False, indent=2)

result_file_path


WindowsPath('D:/jupyter/annihilation_gun_case/media/ocr_results/invoice_example.json')